In [1]:
import yfinance as yf
inda = yf.download("INDA", start="2010-01-01", end="2024-12-31")


YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed


In [2]:
inda

Price,Close,High,Low,Open,Volume
Ticker,INDA,INDA,INDA,INDA,INDA
Date,,,,,
2012-02-03,23.198492,23.198492,23.198492,23.198492,100
2012-02-06,23.016167,23.997240,22.946710,23.997240,4100
2012-02-07,22.807798,22.807798,22.799116,22.799116,2300
2012-02-08,23.059578,23.988559,22.920665,23.988559,3700
2012-02-09,22.998804,23.120353,22.998804,23.120353,2500
...,...,...,...,...,...
2024-12-23,53.270000,53.320000,53.060001,53.240002,5075100
2024-12-24,53.389999,53.389999,53.150002,53.169998,1590000
